In [3]:
from env import DATABASE_URI
from sqlalchemy import create_engine, text, select
from sqlalchemy.orm import Session
import models as dm 
from datetime import datetime
engine = create_engine(DATABASE_URI, echo=False )


In [ ]:
##
#### ADD THERMAL MOCKUP
##

THERMAL_MOCKUP_SERIAL_NUMBER = "ABC123"
with Session(engine) as session:
    thermal_mockup = dm.Module(
        name = THERMAL_MOCKUP_SERIAL_NUMBER
    )
    session.add(thermal_mockup)
    session.commit()

In [ ]:
##
#### SOLDER PLATE UPLOAD
##
with Session(engine) as session:
    # # add Cold Plate configuration (epoxy and solder v1 and solder v2)
    with open("/home/hayden/Downloads/pretty_double_loop_solder_plate.png", 'rb') as f:
        plate_image = f.read()
    plate = dm.ColdPlate(
        name='Pretty Double Loop Solder Plate',
        positions = {
            1:  'front side, top row, far left',
            2:  'front side, top row, middle left',
            3:  'front side, top row, middle right',
            4:  'front side, top row, far right',
            5:  'front side, middle row, far left',
            6:  'front side, middle row, middle left',
            7:  'front side, middle row, middle right',
            8:  'front side, middle row, far right',
            9:  'front side, bottom row, far left',
            10: 'front side, bottom row, middle left',
            11: 'front side, bottom row, middle right',
            12: 'front side, bottom row, far right',
            13: 'back side, top row, far left',
            14: 'back side, top row, middle left',
            15: 'back side, top row, middle right',
            16: 'back side, top row, far right',
            17: 'back side, middle row, far left',
            18: 'back side, middle row, middle left',
            19: 'back side, middle row, middle right',
            20: 'back side, middle row, far right',
            21: 'back side, bottom row, far left bottom',
            22: 'back side, bottom row, middle left bottom',
            23: 'back side, bottom row, middle right bottom',
            24: 'back side, bottom row, far right bottom'
        },
        plate_image = plate_image,
    )
    session.add(plate)
    session.commit()

In [ ]:
##
#### CALIBRATION DATA
## 
import json

MODULE_SERIAL_NUMBER = 'TM0001'
CALIBRATION_DATA_PATH = "/home/hayden/repos/module_pcb_thermal_mockup/module_pcb_thermal_mockup_v3.5/software/CalibrationGUI/calibration_data_TM0001_2024-09-16 13:25:57.037017.json"

ENABLED_SENSORS = ['E1','E3','E4']

CALIBRATION_COMMENT = "This is the recalibration for the thermal mockup that was done by Hayden at IRRAD"

with Session(engine) as session:
    query = select(dm.Module).where(dm.Module.name == MODULE_SERIAL_NUMBER)
    thermal_mockup = session.execute(query).scalars().first()
    
    with open(CALIBRATION_DATA_PATH, 'r') as f:
        calib_data = json.load(f)

    db_calibs = []
    for sensor in ENABLED_SENSORS:
        db_calibs.append(
            dm.SensorCalibration(
                module = thermal_mockup,
                sensor = sensor,
                fit_ohms_over_celcius = calib_data[sensor]['fit_slope'],
                fit_ohms_intercept = calib_data[sensor]['fit_intercept'],
                celcius = calib_data[sensor]['temps'],
                ohms = calib_data[sensor]['ohms'],
                raw_adc = calib_data[sensor]['raw_adcs'],
                times = calib_data[sensor]['times'],
                all_raw_adcs = calib_data[sensor]['all_raw_adcs'],
                all_raw_times = calib_data[sensor]['all_times'],
            )
        )
    session.add_all(db_calibs)
    raise NotImplementedError("FIX the below code for selected sensors :)")
    mod_recalib = dm.ModuleCalibration(
        E1 = db_calibs[0],
        E3 = db_calibs[1],
        E4 = db_calibs[2],
        module = thermal_mockup,
        comment=CALIBRATION_COMMENT
    )

    session.add(mod_recalib)

    thermal_mockup.calibration = mod_recalib

    session.commit()